Cluster the reviews that you collected in homework 5, by doing the following:




##### 1.In Python, select any one of the clustering methods covered in this course. Run it over the collection of reviews, and show at least two different ways of clustering the reviews, e.g., changing k in k-Means clustering or changing where you “cut” in Agnes or Diana.  

In [13]:
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.cluster import KMeans
from bs4 import BeautifulSoup
from requests import get

import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords


url = 'https://www.imdb.com/list/ls095374765/?sort=moviemeter,asc&st_dt=&mode=grid&page=1&ref_=ttls_vw_smp&genres=Comedy'
response = get(url)
html_soupMain = BeautifulSoup(response.text, 'html.parser')

In [15]:
movieReview = []
reviewTitle = []
movieTitle = []

In [16]:
def list_Of_Reviews(website):
    soup = BeautifulSoup(website, 'html.parser')

    f = soup.find_all('div', class_='ipl-icon ipl-star-icon')
    f = soup.find_all('span', class_="rating-other-user-rating")#.find('span').text

    idxPos = -1
    idxNeg = -1

    for idx, i in enumerate(f):
        sp = (f[idx].text.split('/'))
        sp = int(sp[0][-2:])
        if sp > 8:
            idxPos = idx
        elif sp < 4:
            idxNeg = idx
    return([idxNeg,idxPos])

In [17]:
stop_words = set(stopwords.words("english"))
for div in html_soupMain.findAll('div', attrs={'class':'title'}):
    if len(movieReview) > 98:
        break
    else:
        url = 'https://www.imdb.com' + div.find('a')['href'] 
        url = url + 'reviews'
        response = get(url)
        html_soup = BeautifulSoup(response.text, 'html.parser')
        review_containers = html_soup.find_all('div', class_='text show-more__control')
        title_containers = html_soup.find_all( class_='title')
        
        reviewIndex = list_Of_Reviews(response.content)
        reviewList = []
        if reviewIndex[0] == -1 or reviewIndex[1] == -1:
            pass
        else:
            reviewList.append(review_containers[reviewIndex[0]].text)
            reviewList.append(review_containers[reviewIndex[1]].text)
            reviewTitle.append(title_containers[reviewIndex[0]].string + ' -')
            reviewTitle.append(title_containers[reviewIndex[1]].string + ' +')
            movieTitle.append(div.string + ' -')
            movieTitle.append(div.string + ' +')
            count =+ 2
            for i in reviewList:
                review = i.lower()
                #remove punctuation
                review = re.sub('[^a-zA-Z]', ' ', review)
                
                #remove tags
                review = re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",review)

                #remove special characters and digits
                review = re.sub("(\\d|\\W)+"," ",review)

                split_text = review.split()
                
                #Lemmatisation
                lem = WordNetLemmatizer()
                split_text = [lem.lemmatize(word) for word in split_text if not word in stop_words and len(word) >2] 
                split_text = " ".join(split_text)
                movieReview.append(split_text)

In [19]:
print(len(movieReview))

100


In [22]:
#Here we are calling in a TF-IDF vectorizer to transform the data
from sklearn.feature_extraction.text import TfidfVectorizer
tfv = TfidfVectorizer(stop_words = {'english'})#stop_words, ngram_range = (1,1))#transform
vec_text = tfv.fit_transform(movieReview)#returns a list of words.
words = tfv.get_feature_names()
X = vec_text

## Running with K = 8 centriods

In [42]:
import pandas as pd
true_k = 8
model = KMeans(n_clusters=true_k, max_iter=200, n_init=10, tol = 0.001)
model.fit(X)
labels=model.labels_
reviews_cl=pd.DataFrame(list(zip(movieTitle, reviewTitle,labels)),columns=['Movie','Review title','cluster'])
reviews_cl.sort_values(by=['cluster'])

,Movie,Review title,cluster
70,Teen Titans GO! To the Movies -,Teen Titans Go! To the Movies: #cringe\n -,0
51,Zoom +,Dumber Than a Speeding Bullet.\n +,0
46,Inspector Gadget -,"Not as bad as some say, but it could have bee...",0
61,Hobo with a Shotgun +,Hobo With Hauer!\n +,0
34,Spies Like Us -,Chump-comedy with beleaguered leads...\n -,0
...,...,...,...
73,Cocoon: The Return +,More ETs going home\n +,7
16,The 'Burbs -,The *other* (superior) 80s Tom Hanks movie in...,7
81,Deathstalker II +,Slightly better than the first\n +,7
67,Captain Ron +,Not on my boat you don't\n +,7


In [43]:
#here we can see the out put of the 8  k 
simmlar_words = model.cluster_centers_.argsort()[:,-1:-11:-1]
for num, centroid in enumerate(simmlar_words):
    print(str(num) + ' : ' + ', '.join(words[word] for word in centroid))

0 : disney, chase, zoom, chevy, played, movie, audience, superhero, sky, film
1 : film, comedy, movie, one, really, good, bad, see, character, like
2 : movie, film, one, funny, batman, still, character, great, get, effect
3 : murder, mystery, joke, wild, family, ninja, europe, sandler, haru, incredibles
4 : vacation, christmas, loved, vega, bit, family, movie, especially, light, clark
5 : candy, john, rental, summer, film, levy, movie, chet, hear, bos
6 : zorro, movie, good, nice, old, fashioned, justice, fight, thurman, director
7 : first, movie, epic, watching, watch, even, maybe, part, like, parody


## Running with K = 4 centriods

In [44]:
import pandas as pd
true_k = 4
model = KMeans(n_clusters=true_k, max_iter=200, n_init=10, tol = 0.001)
model.fit(X)
labels=model.labels_
reviews_cl=pd.DataFrame(list(zip(movieTitle, reviewTitle,labels)),columns=['Movie','Review title','cluster'])
reviews_cl.sort_values(by=['cluster'])

,Movie,Review title,cluster
14,Howard the Duck -,"Just really, really, really bad\n -",0
63,The Castle +,Best Aussie movie ever\n +,0
23,The Adventures of Sharkboy and Lavagirl 3-D +,"Absolute phenomenal film, LOVED the 3d bluray...",0
21,Super +,Funny yet juvenile\n +,0
36,National Lampoon's European Vacation -,Some funny stuff here!\n -,0
...,...,...,...
4,Weird Science -,"Brings New Meaning to ""Made to Order""\n -",3
35,Spies Like Us +,"Past its sell-by, but still pretty good.\n +",3
44,Summer Rental -,"A prototype for John Candy's future success, ...",3
45,Summer Rental +,Always a welcome visit\n +,3


In [46]:
#ouput for the 4 centroids from reviews

In [45]:
simmlar_words = model.cluster_centers_.argsort()[:,-1:-11:-1]
for num, centroid in enumerate(simmlar_words):
    print(str(num) + ' : ' + ', '.join(words[word] for word in centroid))

0 : movie, humor, fun, funny, like, bad, film, little, probably, batman
1 : film, movie, one, character, first, thing, time, make, kid, like
2 : movie, christmas, vacation, really, family, bit, light, ron, see, moore
3 : candy, john, levy, rental, summer, film, movie, chet, good, hughes


#### 2.	Try to write a short phrase to characterize (give a natural interpretation of) what each cluster is generally centered on semantically. Is this hard to do in some cases? If so, make note of that fact. 



In this assignment I used two kmeans clustering  to be able to cluster together the different  words in the  reviews and have similar reviews join into a cluster. In the  first model the K has been set to 8, which will then produce 8 different centroids. The first cluster is about superheroes  and Disney  that occur in the movie reviews. The second one  had a positive  or negative notation in reviews. The third on seemed to be about movies that  have a  lasting effect on the audience. The fourth cluster was hard to interpret . The fifth cluster is about  movies that are centered around holidays and how that makes sense as there are comedy’s that are about holidays. The sixth cluster is about  movie reviews with other characters that come up during the film and that usually these movies are rentals. The seventh  about  movies that are centered around a character  there is even a character such as zorro that comes up. Lastly  the eight cluster is about  some sub genres in comedy such as parody.

I also did a clustering with 4 k means which allowed the model to make 4 different centroids. The first centroid  has to do with general ideas about the plot of the movie. The  second one is revies about more common review points such as time and how it plays into comedy reviews. The third centroid is about the common holidays that movies can be about. The last one is a centroid that came up similar to  the k means k 8 example this is about and some actors that might have played in them.



#### 3.	Explain which of the two clustering results from question 1 is preferable (if one of them is), and why. Submit all of your inputs and outputs and your code for this assignment, along with a brief written explanation of your findings. 



I would say having the k means  set to 8  k seemed to be a lot more  efficient in a  better in finding the right reviews that associated together. The  k means that was set for only four might have clustering that  is too wide and when you made a EDA you might  not enough information.  This can be readjusted in order to  have to work in different use cases which can be very helpful in the future based on business needs.
